# Init

In [591]:
import pandas as pd

In [592]:
vcfDf=pd.read_csv('/home/jovyan/efs/all_seq/meta_data/All_20170710.f1_byte2_not_00.vcf.gz'
                  ,comment='#',sep='\s+',header=None)

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [593]:
vcfDf.columns=['CHROM', 'POS' ,  'ID' , 'REF' ,'ALT', 'QUAL' , 'FILTER', 'INFO']

In [594]:
vcfDf["REF_len"]=vcfDf['REF'].str.len()

chromToPosRsId=vcfDf.sort_values('REF_len').groupby(['CHROM','POS'])['ID'].first()


In [595]:
mergedDf_rd_reset=pd.read_pickle('mergedDf_rd_reset.pickle')

In [596]:
altBaseDf=mergedDf_rd_reset[mergedDf_rd_reset.AltBase]

In [597]:
refBaseDf=mergedDf_rd_reset[~mergedDf_rd_reset.AltBase]

In [598]:
indexL=['Run_db','Run_digits','Chr','Pos']
refDepth=refBaseDf.set_index(indexL)['ReadDepth']

In [599]:
altBaseDf['RefRD']=refDepth[altBaseDf.set_index(indexL).index].values

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [600]:
altBaseDf['RefRD'].fillna(0,inplace=True)

/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [619]:
altBaseDf['Run_str']=altBaseDf['Run_db'].astype(str)+altBaseDf['Run_digits'].astype(str)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### get per sample altenative allele

In [620]:
altBaseDf_reI=altBaseDf.set_index(['Chr','Pos','base','Run_str'])
altRdFlatDf=altBaseDf_reI['ReadDepth'].unstack().fillna(0)

In [621]:
RefRdFlatDf=altBaseDf_reI['RefRD'].unstack().fillna(0)

In [622]:
AD=RefRdFlatDf.astype(int).astype('str')+':'+altRdFlatDf.astype(int).astype(str)

In [623]:
sampleDf=AD

### generate more info

In [624]:
qualS=altBaseDf_reI.groupby(['Chr','Pos','base'])['AverageBaseQuality'].median()
qualS.name='QUAL'

refBase=altBaseDf_reI.groupby(['Chr','Pos','base'])['Ref'].first()
refBase.name='REF'

combinedDepth=(RefRdFlatDf+altRdFlatDf).sum(axis=1)

poolAF=RefRdFlatDf.sum(axis=1)/(RefRdFlatDf.sum(axis=1)+altRdFlatDf.sum(axis=1))#.fillna(0)
infoS="DP="+combinedDepth.astype(int).astype(str)+';AF='+poolAF.astype(str).str[:6]
infoS.name='INFO'

# Params

In [625]:
thresholdForVars=0.05
thresholdForDepth=10

In [626]:
FilterS=((combinedDepth>thresholdForDepth)
         &(poolAF>thresholdForVars)).map({True:'PASS',
                                              False:'FAIL'})
FilterS.name='FILTER'

In [628]:
mergedDf=pd.concat([qualS,refBase,FilterS,infoS,combinedDepth,sampleDf],axis=1)#.reset_index()

In [629]:
mergedDf.index.names=['#CHROM','POS','ALT']

In [630]:
mergedDf_reI=mergedDf.reset_index()

In [631]:
mergedDf_reI['ID']='-'

In [632]:
mergedDf_reI['FORMAT']='AD'

In [633]:
mandatoryCols=['#CHROM','POS','ID','REF','ALT','QUAL','FILTER','INFO','FORMAT']
mergedDf_reordered=mergedDf_reI[mandatoryCols#
                                +sampleDf.columns.tolist()
                               ]

In [634]:
mergedDf_reordered.to_csv('output_file.vcf',sep='\t',index=None)